folder `dataset` i `predictions` powinny byc w tym samym katalogu, z ktorego sie uruchamia

In [ ]:
# import kagglehub

# #ustaw na swoja
# path = kagglehub.dataset_download("ipythonx/mvtec-ad")

# print("Path to dataset files:", path)

In [3]:
# !mv /root/.cache/kagglehub/datasets/ipythonx/mvtec-ad/versions/2 /content/dataset

In [ ]:
# !pip install anomalib

In [5]:
import torch
import json
import gc
import os
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForCausalLM
from pathlib import Path
from PIL import Image
from tqdm.notebook import tqdm

In [6]:
!ls dataset

bottle	 carpet    leather	pill	    tile	wood
cable	 grid	   license.txt	readme.txt  toothbrush	zipper
capsule  hazelnut  metal_nut	screw	    transistor


dataset

In [7]:
import os
from pathlib import Path

def dataset_generator(root_path, go_to_test=True):
    """
    Generator iterujący po obrazach w folderach 'test' w strukturze MVTec.

    """
    dataset = Path(root_path)
    valid_extensions = {'.png'}

    # 1. Iterujemy po klasach (np. capsule, screw)
    for class_dir in dataset.iterdir():
        if not class_dir.is_dir():
            continue

        if go_to_test:
          test_dir = class_dir / 'test'
          if not test_dir.exists():
              continue

        # 3. Iterujemy po typach defektów/good (to są nasze labele)
        for label_dir in test_dir.iterdir():
            if not label_dir.is_dir():
                continue

            label = label_dir.name

            for image_path in label_dir.iterdir():
                if image_path.suffix.lower() in valid_extensions:
                    # Zwracamy absolutną ścieżkę jako string oraz label
                    yield str(image_path.resolve()), label

mięsko

In [9]:
preds_dir = Path("predictions")
os.makedirs(preds_dir, exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cpu


In [ ]:
def run_llava_eval(question, data_provider, res_dir = preds_dir, res_file_prefix: str = None):
  lava_model_id = "llava-hf/llava-1.5-7b-hf"
  lava_model = LlavaForConditionalGeneration.from_pretrained(
      lava_model_id,
      torch_dtype=torch.float16 if device == "cuda" else torch.float32,
      device_map="auto" if device == "cuda" else None,
      low_cpu_mem_usage=True
  )
  lava_processor = AutoProcessor.from_pretrained(lava_model_id)

  lava_outputs = []

  for i, (path, label) in enumerate(tqdm(dataset_generator('dataset_path'))):
      image = Image.open(path).convert("RGB")

      conversation = [
          {
              "role": "user",
              "content": [
                  {"type": "image", "image": image},
                  {"type": "text", "text": question}
              ],
          }
      ]

      inputs = lava_processor.apply_chat_template(
          conversation,
          add_generation_prompt=True,
          tokenize=True,
          return_dict=True,
          return_tensors="pt"
      )
      inputs = {k: v.to(device) for k, v in inputs.items()}

      input_len = inputs["input_ids"].shape[1]  # 🔑 KLUCZOWE

      with torch.no_grad():
          output_ids = lava_model.generate(
              **inputs,
              max_new_tokens=100,
              do_sample=False
          )

      # ✅ WYCIĄGAMY TYLKO NOWO WYGENEROWANE TOKENY
      answer_ids = output_ids[:, input_len:]

      response = lava_processor.batch_decode(
          answer_ids,
          skip_special_tokens=True
      )[0].strip()

      lava_outputs.append({
          "image_path": path,
          "lava_response": response,
          "label": label
      })

      # if i == 2: break

  lava_results_file = f"{res_file_prefix if res_file_prefix else ""}lava_outputs.json"
  with open(res_dir / lava_results_file, "w") as f:
      json.dump(lava_outputs, f, indent=4)

  print(f"LLaVA outputs saved to {lava_results_file}")

  del lava_model, lava_processor
  gc.collect()
  torch.cuda.empty_cache()

  return res_dir / lava_results_file


In [ ]:
def run_qwen_eval(llava_eval_path, EXTENDED_TEST = False, res_dir = preds_dir, res_file_prefix: str = None):
  with open(llava_eval_path, "r") as f:
      lava_outputs = json.load(f)
  print(f"Loaded {len(lava_outputs)} LLaVA outputs for Qwen processing.")

  qwen_model_name = "Qwen/Qwen2.5-7B-Instruct"
  qwen_tokenizer = AutoTokenizer.from_pretrained(qwen_model_name)
  qwen_model = AutoModelForCausalLM.from_pretrained(
      qwen_model_name,
      device_map="auto" if device == "cuda" else None,
      torch_dtype=torch.float16 if device == "cuda" else torch.float32,
      low_cpu_mem_usage=True
  )

  qwen_results = []

  for lava_out in tqdm(lava_outputs):
      if not EXTENDED_TEST:
        lava_out["label"] = "anomaly" if lava_out["label"] == "good" else "normal"
        prompt = (
            "You are given a condition description of an inspected object.\n\n"
            "Description:\n"
            f"{lava_out['lava_response']}\n\n"
            "Question:\n"
            "Does the description indicate the presence of any defect or anomaly?\n\n"
            "Answer 'yes' if an anomaly is indicated.\n"
            "Answer 'no' if the object appears normal and defect-free.\n"
            "Answer 'yes' or 'no' only, nothing else.\n\n"
            "Answer:"
        )
      else:
        prompt = (
            "You are given a condition description of an inspected object.\n\n"
            "Description:\n"
            f"{lava_out['lava_response'].replace("_", " ").capitalize()}.\n\n"
            "Question:\n"
            "Does the description indicate the presence of any defect or anomaly?\n\n"
            "Answer 'yes' if an anomaly is indicated.\n"
            "Answer 'no' if the object appears normal and defect-free.\n"
            "Answer 'yes' or 'no' only, nothing else.\n\n"
            "Answer:"
        )

      inputs = qwen_tokenizer(prompt, return_tensors="pt").to(device)
      input_len = inputs["input_ids"].shape[1]  # 🔑 KLUCZOWE

      with torch.no_grad():
          output_ids = qwen_model.generate(
              **inputs,
              max_new_tokens=5,
              do_sample=False
          )

      # TYLKO NOWE TOKENY (ODPOWIEDŹ)
      answer_ids = output_ids[:, input_len:]
      response = qwen_tokenizer.decode(
          answer_ids[0],
          skip_special_tokens=True
      ).strip().lower()

      expected = "yes" if lava_out["label"] == "anomaly" else "no"

      if response.startswith(expected):
          prediction = "matches_label"
      elif response.startswith("yes") or response.startswith("no"):
          prediction = "does_not_match_label"
      else:
          prediction = "unknown"

      qwen_results.append({
          "image_path": lava_out["image_path"],
          "lava_response": lava_out["lava_response"],
          "label": lava_out["label"],
          "qwen_answer": response,
          "qwen_prediction": prediction
      })

  # for res in qwen_results:
  #     print("=== IMAGE ===", res["image_path"])
  #     print("LLaVA Output:", res["lava_response"])
  #     print("LABEL:", res["label"])
  #     print("Qwen Answer:", res["qwen_answer"])
  #     print("Parsed:", res["qwen_prediction"])
  #     print("-----------------------------")

  qwen_results_file = f"{res_file_prefix if res_file_prefix else ""}qwen_results{"_extended" if EXTENDED_TEST else ""}.json"
  with open(res_dir/ qwen_results_file, "w") as f:
      json.dump(qwen_results, f, indent=4)

  print(f"Qwen results saved to {qwen_results_file}")

  del qwen_model, qwen_tokenizer
  gc.collect()
  torch.cuda.empty_cache()

  return preds_dir / qwen_results_file


In [ ]:
gc.collect()
torch.cuda.empty_cache()

checking score

In [ ]:
def compute_accuracy(qwen_results_path: str) -> float:
    """
    Computes accuracy as the percentage of entries with
    qwen_prediction == 'matches_label'.
    """
    with open(qwen_results_path, "r") as f:
        results = json.load(f)

    if len(results) == 0:
        print("No results found.")
        return 0.0

    total = len(results)
    correct = sum(
        1 for r in results
        if r.get("qwen_prediction") == "matches_label"
    )

    accuracy = 100.0 * correct / total

    print(f"Accuracy: {accuracy:.2f}% ({correct}/{total})")


## Run experiments

Dwa typy runow:

- **Zwykly test** poleg na potwirdzeniu przez qwen czy llava zauwazym czy jest jakakolwiek anomalia.

- **Extended test** rozszerza to o dodatkowe sprawdzenie czy rodzaj anmoalii sie zgadza; rodzaj jest brany z labela (np. dla pill to beda color, combined, crack, ...)

### First, on a raw dataset test samples:

In [ ]:
question = (
      "Inspect the object for defects like scratches, cracks, fractures, "
      "bending, or any other visible defects. "
      "If found, describe the anomaly ."
      "Otherwise, provide a neutral description. "
  )
llava_res_path = run_llava_eval(question, dataset_generator('dataset', go_to_test=True))

In [ ]:
qwen_res_path = run_qwen_eval(llava_eval_path = llava_res_path, EXTENDED_TEST = False)
qwen_res_path_extended = run_qwen_eval(llava_eval_path = llava_res_path, EXTENDED_TEST = True)

zwykly

In [ ]:
compute_accuracy(qwen_res_path)

extended

In [ ]:
compute_accuracy(qwen_res_path_extended)

### Now, on a WinCLIP anomaly prediciton regions:

In [ ]:
question = (
    "Focusing your attention specifically on the areas enclosed within the two red bounding boxes, "
    "inspect for defects like scratches, cracks, fractures, bending, or any other visible anomalies. "
    "If a defect is found in these areas, describe it. "
    "Otherwise, provide a neutral description indicating the area is clear."
)
llava_res_path = run_llava_eval(dataset_generator('predictions', go_to_test=False))

In [ ]:
qwen_res_path = run_qwen_eval(llava_eval_path = llava_res_path, EXTENDED_TEST = False, res_file_prefix='winclip')
qwen_res_path_extended = run_qwen_eval(llava_eval_path = llava_res_path, EXTENDED_TEST = True, res_file_prefix='winclip')

zwykly

In [ ]:
compute_accuracy(qwen_res_path)

extgended

In [ ]:
compute_accuracy(qwen_res_path_extended)

---

mozna jeszcze wyekstrachowac staty dla psozczegolncyh klas